In [1]:
# STEP 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:

# STEP 2: Load Data
attendance = pd.read_csv("attendance.csv")
train = pd.read_csv("train.csv")
submission = pd.read_csv("submission.csv")


In [ ]:

# STEP 3: Convert datetime + Extract Time Features
attendance['datetime'] = pd.to_datetime(attendance['datetime'])
#attendance['hour'] = attendance['datetime'].dt.hour + attendance['datetime'].dt.minute / 60
#attendance['dayofweek'] = attendance['datetime'].dt.dayofweek
#attendance['day'] = attendance['datetime'].dt.day
#attendance['month'] = attendance['datetime'].dt.month
#attendance['is_weekend'] = attendance['dayofweek'].isin([5,6]).astype(int)


AttributeError: module 'pandas' has no attribute 'to_attendance'

In [ ]:

# STEP 4: Aggregate Features per user_id
user_features = attendance.groupby('user_id').agg({
    'hour': ['mean', 'std', 'min', 'max'],
    'dayofweek': ['mean', 'std'],
    'is_weekend': ['mean'],
    'datetime': ['count', 'nunique']
}).reset_index()

# Rename columns for clarity
user_features.columns = [
    'user_id',
    'hour_mean', 'hour_std', 'hour_min', 'hour_max',
    'dow_mean', 'dow_std',
    'weekend_ratio',
    'total_logs', 'unique_days'
]

# STEP 5: Prepare Training Data
train_df = pd.merge(train, user_features, on='user_id', how='left')
le = LabelEncoder()
train_df['house_label'] = le.fit_transform(train_df['house'])

X_train = train_df.drop(columns=['user_id', 'house', 'house_label'])
y_train = train_df['house_label']

# Fill any NA values
X_train = X_train.fillna(-1)

# STEP 6: Train Model
model = RandomForestClassifier(n_estimators=150, random_state=42)
model.fit(X_train, y_train)

# STEP 7: Prepare Test Data
test_df = pd.merge(submission[['user_id']], user_features, on='user_id', how='left')
X_test = test_df.drop(columns=['user_id'])
X_test = X_test.fillna(-1)

# STEP 8: Predict
y_pred = model.predict(X_test)
pred_houses = le.inverse_transform(y_pred)

# STEP 9: Create submission.csv
submission_out = pd.DataFrame({
    'user_id': test_df['user_id'],
    'house': pred_houses
})
submission_out.to_csv("submission.csv", index=False)

print("✅ เสร็จเรียบร้อย! บันทึก submission.csv แล้ว")
